In [175]:
import os, io, json, urllib, numpy as np, codecs
import pandas as pd

In [176]:
    url="https://api.crossref.org/journals?rows=1"
    response = urllib.urlopen(url).read()
    json_result=json.loads(response)
    count=json_result['message']['total-results']

In [177]:
base_url="https://api.crossref.org/journals?rows=1000&offset="

In [301]:
#f=codecs.open('journals.csv','w','utf-8')
dic=[]
for it in range(count/1000): # change
    print it,
    offset = it * 1000
    url=base_url+str(offset)
    response = urllib.urlopen(url).read()
    json_result=json.loads(response)
    items=json_result['message']['items']
    for item in items:
        if len(item['ISSN'])>0: 
            journal={}
            journal['@id']=item['ISSN'][0]
            if 'title' in item: journal['e:title']=item['title'].strip()
            if 'publisher' in item: journal['e:publisher']=item['publisher'] 
            journal['e:issn']=item['ISSN']
            journal['e:subjects']=[]
            for subject in item['subjects']:
                journal['e:subjects'].append(subject['name'])
            if journal not in dic:
                dic.append(journal)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128


In [302]:
dic_c=dic

In [303]:
with open('road.tsv') as f:
    road_lines = f.read().splitlines()
road_lines=road_lines[1:]

In [304]:
road_urls=[issn[1:-1] for issn in road_lines]
road_issns=[url[-9:] for url in road_urls]

In [305]:
i=0
for journal in dic:
    issn=journal['@id']
    if journal['@id'] in road_issns:
        journal['e:open']=1
        journal['e:road_url']=road_urls[road_issns.index(issn)]
        dic[i]=journal
    i+=1

In [306]:
df=pd.DataFrame(dic)
df.shape

(76670, 7)

In [307]:
df[df['e:issn'].str.len()>0].groupby('e:title').count().shape

(50506, 6)

In [308]:
df[df['e:subjects'].str.len()>0].groupby('e:title').count().shape

(21348, 6)

In [309]:
df

,@id,e:issn,e:open,e:publisher,e:road_url,e:subjects,e:title
0,0048-2129,"[0048-2129, 2366-0481]",NaN,Walter de Gruyter GmbH,NaN,[],ORDO
1,0029-8948,"[0029-8948, 2199-6881]",NaN,Springer-Verlag,NaN,"[Ecology, Evolution, Behavior and Systematics,...",Österreichische Botanische Zeitschrift
2,0030-7270,"[0030-7270, 2043-6866]",NaN,SAGE Publications,NaN,"[Agronomy and Crop Science, Animal Science and...",Outlook on Agriculture
3,1609-9451,"[1609-9451, 1793-687X]",NaN,World Scientific,NaN,"[Civil and Structural Engineering, Building an...",Journal of Restructuring Finance
4,2356-6590,[2356-6590],NaN,Hindawi Limited,NaN,[],Advances in Biomaterials
5,0976-4585,[0976-4585],1.0,Society of Education,http://road.issn.org/issn-l/0976-4585,[],Advances in Bioresearch
6,1517-9702,[1517-9702],1.0,SciELO,http://road.issn.org/issn-l/1517-9702,[Education],Educação e Pesquisa
7,1979-7281,[1979-7281],NaN,E-Journal System Portal of Syarif Hidayatullah...,NaN,[],EDUSAINS
8,0066-4774,"[0066-4774, 0066-4774]",NaN,Cambridge University Press,NaN,[Classics],Antichthon
9,2249-1449,[2249-1449],NaN,A.J. Institute of Management,NaN,[],Anveshana search for Knowledge


In [312]:
batches=len(dic)/10000
for s in range(batches+1):
    dic_chunk=dic[10000*s:10000*(s+1)]
    with open('jsonld/'+str(s)+'journals.jsonld', 'w') as outfile:
        json.dump(dic_chunk, outfile)

In [184]:
df.to_csv('journals.csv',encoding='utf-8')

In [185]:
unique_subjects=[]
for ss in df['e:subjects'].values:
    for s in ss:
        if s not in unique_subjects: unique_subjects.append(s)
print len(unique_subjects)

330


In [186]:
pd.DataFrame(unique_subjects).to_csv('subjects.csv',index=False,header= False, encoding='utf-8')

In [205]:
with open ('ASJC.txt','r') as myfile:
    ASJC=myfile.read()

In [207]:
ASJC=ASJC.replace('General','Multidisciplinary')
ASJC=ASJC.replace('veterinary','Veterinary')
ASJC=ASJC.replace('General veterinary','General Veterinary')
ASJC=ASJC.replace('veterinary (miscalleneous','Veterinary (miscalleneous)')

In [208]:
from StringIO import StringIO
subject_cats=pd.read_table(StringIO(ASJC),delimiter=';')#from github
subject_cats.shape 
subject_cats['Low']=subject_cats.Low.apply(lambda x: 'General '+x.replace('(all)','') if '(all)' in x else x)

In [213]:
for i in subject_cats.Low.values:
    if i not in unique_subjects:
        print i

Medical–Surgical
Pathophysiology
Medical Terminology
Respiratory Care


C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


In [214]:
for i in unique_subjects:
    if i not in subject_cats.Low.values:
        print i

Medical–Surgical


C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


In [215]:
cats={}
for i, row in subject_cats.iterrows():
    cat=row.Middle.strip()
    subject=(row.Low).strip()
    if cat in cats:
        cats[cat]=cats[cat]+';'+subject
    else:
        cats[cat]=subject

In [216]:
Middles=cats.keys()
Middles.sort()

In [217]:
for key in Middles:
    print '<option value="'+key+'">'+key+'</option>'

<option value="Agricultural and Biological Sciences">Agricultural and Biological Sciences</option>
<option value="Arts and Humanities">Arts and Humanities</option>
<option value="Biochemistry, Genetics and Molecular Biology">Biochemistry, Genetics and Molecular Biology</option>
<option value="Business, Management and Accounting">Business, Management and Accounting</option>
<option value="Chemical Engineering">Chemical Engineering</option>
<option value="Chemistry">Chemistry</option>
<option value="Computer Science">Computer Science</option>
<option value="Decision Sciences">Decision Sciences</option>
<option value="Dentistry">Dentistry</option>
<option value="Earth and Planetary Sciences">Earth and Planetary Sciences</option>
<option value="Economics, Econometrics and Finance">Economics, Econometrics and Finance</option>
<option value="Energy">Energy</option>
<option value="Engineering">Engineering</option>
<option value="Environmental Science">Environmental Science</option>
<option va

In [218]:
for key in Middles:
	print 'case "'+key +'":'
	print '\tvals = data["'+key+'"].split(";");'
	print '\tbreak;'

case "Agricultural and Biological Sciences":
	vals = data["Agricultural and Biological Sciences"].split(";");
	break;
case "Arts and Humanities":
	vals = data["Arts and Humanities"].split(";");
	break;
case "Biochemistry, Genetics and Molecular Biology":
	vals = data["Biochemistry, Genetics and Molecular Biology"].split(";");
	break;
case "Business, Management and Accounting":
	vals = data["Business, Management and Accounting"].split(";");
	break;
case "Chemical Engineering":
	vals = data["Chemical Engineering"].split(";");
	break;
case "Chemistry":
	vals = data["Chemistry"].split(";");
	break;
case "Computer Science":
	vals = data["Computer Science"].split(";");
	break;
case "Decision Sciences":
	vals = data["Decision Sciences"].split(";");
	break;
case "Dentistry":
	vals = data["Dentistry"].split(";");
	break;
case "Earth and Planetary Sciences":
	vals = data["Earth and Planetary Sciences"].split(";");
	break;
case "Economics, Econometrics and Finance":
	vals = data["Economics, Econo

In [219]:
import json
with open('sjdb\cats.json', 'w') as outfile:
    json.dump(cats, outfile)

In [230]:
df[df['e:title']=='Nature Nanotechnology']

,@id,e:issn,e:publisher,e:subjects,e:title
28662,NaN,[],58809,[],Nature Nanotechnology
72969,1748-3387,[1748-3387],Nature Publishing Group,"[Bioengineering, Biomedical Engineering, Elect...",Nature Nanotechnology
113393,1748-3387,"[1748-3387, 1748-3395]",Nature Publishing Group,"[Bioengineering, Biomedical Engineering, Elect...",Nature Nanotechnology


In [260]:
df

,@id,e:issn,e:publisher,e:subjects,e:title
0,NaN,[],JScholar,[],JNSM
1,NaN,[],Elsevier,[],New Comprehensive Biochemistry
2,NaN,[],301175,[],New Frontiers in Ophthalmology
3,NaN,[],300049,[],Oral Health Case Reports
4,NaN,[],300942,[],Orbit A Journal of American Literature
5,0048-2129,"[0048-2129, 2366-0481]",Walter de Gruyter GmbH,[],ORDO
6,0029-8948,"[0029-8948, 2199-6881]",Springer-Verlag,"[Ecology, Evolution, Behavior and Systematics,...",Österreichische Botanische Zeitschrift
7,0030-7270,"[0030-7270, 2043-6866]",SAGE Publications,"[Agronomy and Crop Science, Animal Science and...",Outlook on Agriculture
8,NaN,[],University of Toronto Press Inc,[],International Journal of Feminist Approaches t...
9,2155-9546,[2155-9546],OMICS Publishing Group,[],Journal of Aquaculture Research & Development
